In [1]:
import pandas as pd
from copy import deepcopy
from collections import Counter
from sklearn.model_selection import train_test_split

rawdata = pd.read_excel('./rawdata.xlsx') # 720 -> 628 -> 590 -> 567
rawdata = rawdata[['gene','Gene_ID','Promoter_ID','RBS_ID','Promoter_Seq','RBS_Seq','Gene_180bp','平均flu/OD']]
rawdata['平均flu/OD'] = pd.to_numeric(rawdata['平均flu/OD'], errors='coerce')
rawdata['平均flu/OD'] = rawdata['平均flu/OD'].astype(float) + 3000
rawdata = rawdata.dropna()
print(len(rawdata))
rawdata = rawdata[rawdata['Promoter_Seq'].apply(lambda x: 20 < len(x) < 100)] # 683 -> 580
print(len(rawdata))
rawdata = rawdata[rawdata['RBS_Seq'].apply(lambda x: 10 < len(x) < 50)] # 683 -> 580
print(len(rawdata))
rawdata = rawdata[~rawdata.gene.str.contains("_WT")] 
print(len(rawdata))

# P 20 <100 R 10 < 50 CDS ==180
rawdata.index = range(len(rawdata))
rawdata # 720 总数 -> 628 去空-> 590 去WT -> 567 去处异常长度

628
567
567
567


,gene,Gene_ID,Promoter_ID,RBS_ID,Promoter_Seq,RBS_Seq,Gene_180bp,平均flu/OD
0,atoB(thl)_100,atoB(thl),BBa_J23118,apFAB923,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG,ATCTTAATCTAGCGTGGGAGAGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,119164.059432
1,atoB(thl)_130,atoB(thl),apFAB47,B0032_RBS,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,TCACACAGGAAAGGCCCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,164210.862842
2,atoB(thl)_140,atoB(thl),apFAB55,apFAB876,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCTGGC,ATCTTAATCTAGCTAGGGATATTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,14266.357622
3,atoB(thl)_150,atoB(thl),pTrc*,apFAB912,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,ATCTTAATCTAGCTAGGGAGTGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,104460.286090
4,atoB(thl)_160,atoB(thl),BBa_J23100,apFAB918,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT,ATCTTAATCTAGCTAGGGAACGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,20202.330833
...,...,...,...,...,...,...,...,...
562,GFP_320,GFP,apFAB75,JBEI_RBS,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAGTG,AAAGATCTTTTAAGAAGGAGATATACATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,24476.305921
563,GFP_40,GFP,apFAB57,apFAB904,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCCGAC,ATCTTAATCTAGCGGGGGATTTTTTCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,8748.363110
564,GFP_50,GFP,BBa_J23119,BBa_J61129,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA,TCTAGAGAAAGATTGGACGTACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,5553.358256
565,GFP_70,GFP,apFAB46,BBa_J61113,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,TCTAGAGAAAGAGTGGAAAAACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,8146.421157


In [2]:
atdata = deepcopy(rawdata)
atdata['Seq'] = atdata['Promoter_Seq']+atdata['RBS_Seq']+atdata['Gene_180bp']
atdata['Seq'] = atdata['Seq'].apply(lambda x: x.upper())
# 使用apply和lambda函数筛选出长度大于等于250的行
# atdata = atdata[atdata['Seq'].apply(lambda x: 250 < len(x) < 300)] # 683 -> 580
# atdata['Seq'] = atdata['Seq'].apply(lambda x: x[:250])
atdata['平均flu/OD'] = atdata['平均flu/OD'].apply(np.log1p)
# 使用正则表达式删除非"ATCG"的字符
atdata['Seq'] = atdata['Seq'].str.replace(r'[^ATCG]', '')
atdata = atdata.dropna()

atdata = atdata[['Gene_ID','Promoter_ID','RBS_ID','Seq','平均flu/OD']]
atdata.columns = ['Gene','Promoter','RBS','Seq','OD']
atdata.Seq = atdata.Seq.apply(lambda x:x[:248])
atdata

<IPython.core.display.Javascript object>

,Gene,Promoter,RBS,Seq,OD
0,atoB(thl),BBa_J23118,apFAB923,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG...,11.688265
1,atoB(thl),apFAB47,B0032_RBS,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,12.008913
2,atoB(thl),apFAB55,apFAB876,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCT...,9.565730
3,atoB(thl),pTrc*,apFAB912,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,11.556572
4,atoB(thl),BBa_J23100,apFAB918,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT...,9.913603
...,...,...,...,...,...
562,GFP,apFAB75,JBEI_RBS,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAG...,10.105502
563,GFP,apFAB57,apFAB904,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCC...,9.076736
564,GFP,BBa_J23119,BBa_J61129,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA...,8.622338
565,GFP,apFAB46,BBa_J61113,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,9.005457


In [3]:
atdata.Seq.apply(len).describe()

count    567.0
mean     248.0
std        0.0
min      248.0
25%      248.0
50%      248.0
75%      248.0
max      248.0
Name: Seq, dtype: float64

In [4]:
for col in ["Gene","Promoter","RBS"]:
    # 获取唯一值和计数
    unique_cls = atdata[col].unique()
    cls_counts = Counter(atdata[col])

    # 计算需要分成的组数
    total_count = sum(cls_counts.values())
    num_groups = total_count // 5 + 1 if total_count % 5 != 0 else total_count // 5

    # 创建一个空的 dictionary 来存储分组结果
    groups = {i: [] for i in range(1, num_groups + 1)}

    # 遍历唯一值并将其添加到对应的组中
    current_group = 1
    for c, count in cls_counts.items():
        for _ in range(count):
            groups[current_group].append(c)
            if len(groups[current_group]) == num_groups:
                current_group += 1

    # 创建一个映射字典
    mapping = {fruit: group_num for group_num, fruits in groups.items() for fruit in fruits}

    # 将分组结果添加到原DataFrame
    atdata['group'] = atdata[col].map(mapping)

    for i in range(5):
        train = atdata[atdata['group'] != i+1]
        test = atdata[atdata['group'] == i+1]
        train = train[['Seq','OD']]
        train.columns = ['seq','target']
        test = test[['Seq','OD']]
        test.columns = ['seq','target']
        train.to_csv(f"rigorous/train_{col}_group{i+1}.csv",index=False)
        test.to_csv(f"rigorous/test_{col}_group{i+1}.csv",index=False)
        

In [5]:
for col in ["Gene","Promoter","RBS"]:
    for i in range(1,6):
        print(f"rigorous/train_{col}_group{i}.csv")

rigorous/train_Gene_group1.csv
rigorous/train_Gene_group2.csv
rigorous/train_Gene_group3.csv
rigorous/train_Gene_group4.csv
rigorous/train_Gene_group5.csv
rigorous/train_Promoter_group1.csv
rigorous/train_Promoter_group2.csv
rigorous/train_Promoter_group3.csv
rigorous/train_Promoter_group4.csv
rigorous/train_Promoter_group5.csv
rigorous/train_RBS_group1.csv
rigorous/train_RBS_group2.csv
rigorous/train_RBS_group3.csv
rigorous/train_RBS_group4.csv
rigorous/train_RBS_group5.csv
